In [1]:
from document_processor import DocumentProcessor
from knowledge_graph import KnowledgeGraph
from vector_store import VectorStore
from entity_extraction import GraphEntityRetriever
from retrieval import Retriever
from rag_chain import RagChain
from dotenv import load_dotenv
import os
import warnings
warnings.filterwarnings("ignore")
load_dotenv()

required_vars = ["NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD", "OPENROUTER_API_KEY"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    print(f"Warning: The following environment variables are not set: {', '.join(missing_vars)}")
    print("Please set them in a .env file or directly in the environment.")
else:
    print("All required environment variables are set.")

All required environment variables are set.


In [2]:
# Initialize the System

document_processor = DocumentProcessor()
knowledge_graph = KnowledgeGraph()
vector_store = VectorStore(knowledge_graph)
entity_extractor = GraphEntityRetriever()
retriever = Retriever(knowledge_graph, vector_store)
rag_chain = RagChain(retriever)



# Load the documents

In [3]:
documents = document_processor.process_documents()
print(f"Loaded {len(documents)} document chunks from the knowledge base")
print("\nFirst document chunk:")
print(documents[0].page_content[:500] + "...")

Loaded 84 document chunks from the knowledge base

First document chunk:
**Chris Olande: A Deep Professional Profile for RAG Systems**

**Overview**

Chris Olande is an ambitious and intellectually gifted data science student at Kenyatta University in Nairobi, Kenya. With a rigorous academic foundation in statistics and programming, Chris is cultivating a multidisciplinary approach that intersects applied mathematics, artificial intelligence, and human-centered design. He brings together analytical precision, computational depth, and social responsibility in his work...


# Build the knowledge graph

In [4]:
# Clear the existing database
knowledge_graph.clear_database()

# Create a knowledge graph from the documents
knowledge_graph.create_graph_from_documents(documents)

print("Knowledge graph created successfully!")

Extracting entities and relationships from documents...


Processing documents:   0%|          | 0/84 [00:00<?, ?it/s]

Extracted 455 unique entities and 1224 relationships
Creating entities...


Creating entities in batches:   0%|          | 0/5 [00:00<?, ?it/s]

Creating relationships...


...

Creating WEBSITE relationships:   0%|          | 0/1 [00:00<?, ?it/s]

Creating DONATIONS_TAX_DEDUCTIBLE relationships:   0%|          | 0/1 [00:00<?, ?it/s]

Creating COMPLIES_WITH_LAWS_REGULATING_CHARITIES_AND_CHARITABLE_DONATIONS relationships:   0%|          | 0/1 …

Creating DONATION_METHODS_AND_ADDRESSES_AVAILABLE_AT relationships:   0%|          | 0/1 [00:00<?, ?it/s]

Graph creation completed!
Knowledge graph created successfully!


In [5]:
knowledge_graph.visualize_graph()

GraphWidget(layout=Layout(height='770px', width='100%'))

# Test entity Extraction

In [6]:
question = "Who is Chris Olande?"
entity = entity_extractor.extract_entities(question)
print(entity)

Extracting entities for question: Who is Chris Olande?
Extracted 1 entities: ['Chris Olande']
['Chris Olande']


# Test structured entity extraction

In [7]:
structured_results = entity_extractor.structured_retriever(question)

print(f"Question: {question}")
print("\nStructured retrieval results:")
print(structured_results)

Processing structured retrieval for question: Who is Chris Olande?
Extracting entities for question: Who is Chris Olande?
Extracted 1 entities: ['Chris Olande']
Retrieved 50 relationships for entity 'Chris Olande'
Found 50 relationships for entity 'Chris Olande'
  - Chris Olande - USES -> Pinecone
  - Chris Olande - USES -> OpenCV
  - Chris Olande - USES -> vision transformers
  - Chris Olande - USES -> Flask
  - Chris Olande - USES -> Docker
  ... and 45 more
Question: Who is Chris Olande?

Structured retrieval results:
Chris Olande - USES -> Pinecone
Chris Olande - USES -> OpenCV
Chris Olande - USES -> vision transformers
Chris Olande - USES -> Flask
Chris Olande - USES -> Docker
Chris Olande - USES -> Fastai
Chris Olande - USES -> FAISS
Chris Olande - USES -> Streamlit
Chris Olande - USES -> Gradio
Chris Olande - USES -> Python
Chris Olande - USES -> torchvision transforms
Chris Olande - DESIGNS -> intelligent job search assistant
Chris Olande - PROMOTES -> STEM exposure
Chris Oland

# Create the hybrid vector index

In [8]:
vector_store.create_hybrid_index()

print("Hybrid vector index created successfully!")

Hybrid vector index created successfully!


# Create vector index

In [9]:
# create vector index from documents
vector_store.create_vector_index(documents)

print("Vector index created successfully!")

Vector index created successfully!


## Test vector Retrieval

In [10]:
# Test vector retrieval
vector_results = retriever.vector_retrieval(question, k=2)

print(f"Question: {question}")
print("\nVector retrieval results:")
for i, doc in enumerate(vector_results):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:300] + "...")

Question: Who is Chris Olande?

Vector retrieval results:

Document 1:
**Chris Olande: A Deep Professional Profile for RAG Systems**

**Overview**

Chris Olande is an ambitious and intellectually gifted data science student at Kenyatta University in Nairobi, Kenya. With a rigorous academic foundation in statistics and programming, Chris is cultivating a multidisciplina...

Document 2:
 tutoring peers in statistics and machine learning, explaining complex concepts like backpropagation, gradient descent, and dimensionality reduction using intuitive analogies and real-life examples.

**Personality, Work Ethic, and Personal Interests**

Chris is disciplined, curious, and mathematical...

Document 3:
.
Have we anything new to offer upon the subject?  Nothing.  We have held the
subject up in every light of which it is capable; but it has been all in vain.
Shall we resort to entreaty and humble supplication?  What terms shall we
find which have not been already exhausted?  Let us not, I besee

# Test hybrid retrieval


In [11]:
# Test hybrid retrieval
hybrid_results = retriever.hybrid_retrieval(question)

print(f"Question: {question}")
print("\nHybrid retrieval results:")
print(hybrid_results)

Question: Who is Chris Olande?

Hybrid retrieval results:
Structured data:
            No relevant information found in the knowledge graph.

            Unstructured data:
            **Chris Olande: A Deep Professional Profile for RAG Systems**

**Overview**

Chris Olande is an ambitious and intellectually gifted data science student at Kenyatta University in Nairobi, Kenya. With a rigorous academic foundation in statistics and programming, Chris is cultivating a multidisciplinary approach that intersects applied mathematics, artificial intelligence, and human-centered design. He brings together analytical precision, computational depth, and social responsibility in his work. Chris is not only building machine learning systems—he is actively shaping the future of AI literacy and ethical, context-aware data applications in Kenya and beyond.

**Academic Background and Technical Expertise**

Chris’s academic journey is rooted in mathematical reasoning and statistical theory, reinforced 

# Test RAG Chain

In [12]:
# Test the RAG chain
answer = rag_chain.invoke({"question": question})

print(f"Question: {question}")
print(f"\nAnswer: {answer}")

Question: Who is Chris Olande?

Answer:  Chris Olande is an ambitious and intellectually gifted data science student at Kenyatta University in Nairobi, Kenya. He has a rigorous academic foundation in statistics and programming, and is cultivating a multidisciplinary approach that intersects applied mathematics, artificial intelligence, and human-centered design. Chris is not only building machine learning systems, but also actively shaping the future of AI literacy and ethical, context-aware data applications in Kenya and beyond.


## Test using a follow up question

In [13]:
# Initialize chat history
chat_history = [(question, answer)]

# Ask a follow-up question
follow_up_question = "What does he even do at Kenyatta University?"

# Get answer to follow-up question
follow_up_answer = rag_chain.invoke({
    "question": follow_up_question,
    "chat_history": chat_history
})

print(f"Follow-up question: {follow_up_question}")
print(f"\nAnswer: {follow_up_answer}")

Follow-up question: What does he even do at Kenyatta University?

Answer:  Chris Olande is a data science student at Kenyatta University in Nairobi, Kenya. He tutors peers in statistics and machine learning, explaining complex concepts using intuitive analogies and real-life examples. He is also known for his leadership in educational initiatives, promoting STEM exposure and hands-on industry engagement among young learners.


In [14]:
quiz = "How and what tools does he use and what jobs is he suited for in the future?"

# Get answer to follow-up question
follow_up_answer = rag_chain.invoke({
    "question": quiz,
    "chat_history": chat_history
})

print(f"Follow-up question: {quiz}")
print(f"\nAnswer: {follow_up_answer}")

Follow-up question: How and what tools does he use and what jobs is he suited for in the future?

Answer:  Chris Olande uses a variety of tools in data science, including Python, R, Git/GitHub, shell scripting, data science libraries such as NumPy, pandas, matplotlib, seaborn, scikit-learn, SciPy, deep learning and NLP libraries like PyTorch, Hugging Face Transformers, Fastai, OpenCV, LangChain, SentenceTransformers, ML experiment tracking tools like Weights & Biases and TensorBoard, and deployment tools like Streamlit, Gradio, Flask, Docker. Given his background, he might be suited for job opportunities in AI research labs, AI startups, contributing to open-source projects, developing intelligent agents, AI in education, and building systems that personalize learning, assess student understanding, and provide adaptive feedback.


# Test questions based on romeo and juliet knowledge base

In [15]:
question = "give me a brief story about the romeo and juliet book"
# Test the RAG chain
answer = rag_chain.invoke({"question": question})

print(f"Question: {question}")
print(f"\nAnswer: {answer}")

Question: give me a brief story about the romeo and juliet book

Answer:  "Romeo and Juliet" is a tragic play written by William Shakespeare. The story is set in Verona, Italy, and revolves around two young lovers, Romeo Montague and Juliet Capulet, who are from two long-standing feuding families. Despite their families' hatred for each other, Romeo and Juliet fall in love at first sight and secretly marry with the help of Friar Lawrence. However, their marriage is short-lived due to a series of unfortunate events, including the death of Tybalt, a Capulet, at the hands of Romeo, and the subsequent exile of Romeo. In a desperate attempt to reunite the families and save Romeo from exile, Friar Lawrence gives Juliet a potion that will make it appear as if she is dead for 42 hours, allowing Romeo to believe she is dead and giving them time to escape. However, a misunderstanding leads to Juliet being awakened prematurely, and she is found by Romeo, who, believing she is dead, takes poison a

In [16]:
question = "How did Romeo meet Juliet?"

# Get answer to follow-up question
follow_up_answer = rag_chain.invoke({
    "question": question,
    "chat_history": chat_history
})

print(f"Follow-up question: {question}")
print(f"\nAnswer: {follow_up_answer}")

Follow-up question: How did Romeo meet Juliet?

Answer:  Romeo and Juliet first met at a public place where a ball was being held. They met secretly because they were from rival families, Montagues and Capulets, who were at feud with each other. Romeo was disguised as a servant, and Juliet was talking with her nurse. They met and fell in love at first sight.


In [17]:
quiz = "How did Romeo meet his fate? go on to tell me how Juliet met her too."

# Get answer to follow-up question
follow_up_answer = rag_chain.invoke({
    "question": quiz,
    "chat_history": chat_history
})

print(f"Follow-up question: {quiz}")
print(f"\nAnswer: {follow_up_answer}")

Follow-up question: How did Romeo meet his fate? go on to tell me how Juliet met her too.

Answer:  Romeo met his fate by drinking a poison that he believed was a remedy, but it was actually a fatal dose. Juliet met her fate by taking Romeo's dagger and stabbing herself after finding him dead.


# Test for questions about the declaration of independence

In [18]:
question = "what grievances were made against the King of England?"
# Test the RAG chain
answer = rag_chain.invoke({"question": question})

print(f"Question: {question}")
print(f"\nAnswer: {answer}")

Question: what grievances were made against the King of England?

Answer:  The grievances made against the King of England, as stated in the Declaration of Independence, include:

1. Refusing to give assent to laws that were wholesome and necessary for the public good.
2. Forbidding governors to pass laws of immediate and pressing importance without suspending them until his assent was obtained, and neglecting to attend to them when suspended.
3. Refusing to pass laws for the accommodation of large districts of people unless they would relinquish the right of representation in the legislature.
4. Calling legislative bodies together at places unusual, uncomfortable, and distant from the depository of their public records, for the sole purpose of fatiguing them into compliance with his measures.
5. Dissolving representative houses repeatedly for opposing his invasions on the rights of the people.
6. Refusing for a long time to cause others to be elected after dissolutions, leaving the le

In [20]:
quiz = "How did the declaration change the status of the American colonies?"

# Get answer to follow-up question
follow_up_answer = rag_chain.invoke({
    "question": quiz,
    "chat_history": chat_history
})

print(f"Follow-up question: {quiz}")
print(f"\nAnswer: {follow_up_answer}")

Follow-up question: How did the declaration change the status of the American colonies?

Answer:  The Declaration of Independence declared the American colonies to be free and independent states, absolved from all allegiance to the British Crown, and dissolved all political connection between them and the State of Great Britain. This effectively ended the political relationship between the American colonies and Great Britain, and established the United States as a separate and independent nation.


In [21]:
quiz = "What enlightenment ideas are embedded in the declaration of independence?"

# Get answer to follow-up question
follow_up_answer = rag_chain.invoke({
    "question": quiz,
    "chat_history": chat_history
})

print(f"Follow-up question: {quiz}")
print(f"\nAnswer: {follow_up_answer}")

Follow-up question: What enlightenment ideas are embedded in the declaration of independence?

Answer:  The Declaration of Independence, written by Thomas Jefferson, is a foundational document of the United States and embodies several Enlightenment ideas. Here are some key Enlightenment ideas that can be found in the Declaration:

1. Natural Rights: The Declaration asserts that all men are created equal and are endowed by their Creator with certain unalienable Rights, such as Life, Liberty, and the pursuit of Happiness. This idea is rooted in the Enlightenment belief in natural rights, which are inherent to all individuals and cannot be taken away by any government.

2. Consent of the governed: The Declaration states that governments are instituted among men to secure these rights, deriving their just powers from the consent of the governed. This idea reflects the Enlightenment belief in the social contract, whereby individuals agree to be governed in exchange for protection of their r

In [ ]:
quiz = "What were the contradictions in the declaration regarding slavery and equality?"

# Get answer to follow-up question
follow_up_answer = rag_chain.invoke({
    "question": quiz,
    "chat_history": chat_history
})

print(f"Follow-up question: {quiz}")
print(f"\nAnswer: {follow_up_answer}")